In [ ]:
import os
import re

from collections import Counter
from copy import copy

import numpy as np

from tqdm import tqdm

In [ ]:
DATA_PATH = 'D:/MADE/2021/(ML) Продвинутое машинное обучение/2/data'

with open(os.path.join(DATA_PATH, 'AnnaKarenina.txt'), 'r', encoding='utf-8') as f1, \
     open(os.path.join(DATA_PATH, 'WarAndPeace.txt'), 'r', encoding='utf-8') as f2:
    anna_karenina = f1.read().lower()
    war_and_peace = f2.read().lower()

In [ ]:
ALPHABET = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '

Подготовка текста

In [ ]:
def prepare_text(text):
    prepeared = ''
    for l in text.lower():
        if l in ALPHABET:
            prepeared += ''.join(l)
    return(prepeared)

Формируем обучающую выборку

In [ ]:
train_texts = prepare_text(anna_karenina + war_and_peace)

1. Базовый частотный метод

1. Реализуйте базовый частотный метод по Шерлоку Холмсу:


*   подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто 
опустить, а вот пробелы лучше оставить);
*   возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
*   расшифруйте их таким частотным методом.

In [ ]:
def base_freqs(text):
    freqs = {}
    for symbol in text:
        if symbol.lower() not in freqs:
            freqs[symbol.lower()] = 1
        else:
            freqs[symbol.lower()] += 1

    prob = {}
    for k, v in freqs.items():
        prob[k] = v / sum(freqs.values())

    prob = dict(sorted(prob.items(), key=lambda x: x[1], reverse=True))
    return prob

In [ ]:
unigram_freqs = base_freqs(train_texts)

In [ ]:
def encoded(orig_freqs):
    return dict(zip(orig_freqs, np.random.permutation(list(orig_freqs))))

In [ ]:
encode_mapping = encoded(unigram_freqs)

In [ ]:
test_text = """
Она улыбнулась на его похвалу и через его плечо продолжала разглядывать залу. 
Она была не вновь выезжающая, у которой на бале все лица сливаются в одно волшебное впечатление; 
она и не была затасканная по балам девушка, которой все лица бала так знакомы, что наскучили; 
но она была на середине этих двух, — она была возбуждена, а вместе с тем обладала собой настолько, 
что могла наблюдать.  
"""

In [ ]:
filtered_text = prepare_text(test_text)

In [ ]:
def base_decode(text, mapping):
    result = ''
    for l in text:
        result += ''.join(mapping[l])
    return result

In [ ]:
encoded_text = base_decode(filtered_text, encode_mapping)

In [ ]:
text_unigram_freqs = base_freqs(encoded_text)

In [ ]:
def decoded(orig_freqs, text_freqs):
    return dict(zip(text_freqs, orig_freqs))

In [ ]:
decode_mapping = decoded(unigram_freqs, text_unigram_freqs)

In [ ]:
decoded_text = base_decode(encoded_text, decode_mapping)

#Результат:

In [ ]:
print(filtered_text)

она улыбнулась на его похвалу и через его плечо продолжала разглядывать залу она была не вновь выезжающая у которой на бале все лица сливаются в одно волшебное впечатление она и не была затасканная по балам девушка которой все лица бала так знакомы что наскучили но она была на середине этих двух  она была возбуждена а вместе с тем обладала собой настолько что могла наблюдать  


In [ ]:
print(decoded_text)

ено камлнкаовг но изе ыежтоак р яиьип изе ыаияе ыьедеайоао ьопзачдмтосг поак ено лмао ни тнетг тмипйошцоч к уесеьех но лоаи тви арюо вартошсвч т едне теаэилнеи тыияосаинри ено р ни лмао посовуонноч ые лоаоб диткэуо уесеьех тви арюо лоао соу пноуебм ясе новукярар не ено лмао но виьидрни щсрж дткж  ено лмао теплкйдино о тбивси в сиб елаодоао велех новсеагуе ясе безао нолашдосг  


Получилось нечитабельно. Похоже на песню, слова к которой ты не знаешь, но пытаешься петь.

In [ ]:
def accuracy(orig_text, decoded_text):
    return sum(np.array(list(orig_text)) == np.array(list(decoded_text))) / len(orig_text)

In [ ]:
print(f'Доля верно расшифрованных символов: {accuracy(filtered_text, decoded_text):.4f}')

Доля верно расшифрованных символов: 0.2612


#2. Расшифровка через биграммы


*   подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
*   проведите тестирование аналогично п.1, но при помощи биграмм.



In [ ]:
def bigram_freqs(text):
    
    list_of_bigrams = []
    
    for i in range(len(text) - 2):
        list_of_bigrams.append(text[i:i+2])
        
    freqs = Counter(list_of_bigrams)
    prob = {}
    
    for k, v in freqs.items():
        prob[k] = v / sum(freqs.values())
    

    freqs = dict(sorted(prob.items(), key=lambda x: x[1], reverse=True))
    return freqs

def bigram_decode(text, mapping):
    UNKNOWN = '?'
    decoded_text = [UNKNOWN] * len(text) 
    for text_bi, orig_bi in mapping.items():
        index = text.find(text_bi)
        while index != -1:
            if decoded_text[index] == UNKNOWN:
                decoded_text[index] = orig_bi[0]
            if decoded_text[index + 1] == UNKNOWN:
                decoded_text[index + 1] = orig_bi[1] 
            index = text.find(text_bi, index + 1)

    return ''.join(decoded_text)

In [ ]:
bigram_freqs = bigram_freqs(train_texts)

In [ ]:
text_bigram_freqs = bigram_freqs(encoded_text)
decode_mapping = decoded(bigram_freqs, text_bigram_freqs)

decoded_text = bigram_decode(encoded_text, decode_mapping)

In [ ]:
print(f'Доля верно расшифрованных символов: {accuracy(filtered_text, decoded_text):.4f}')

Доля верно расшифрованных символов: 0.1557


#Результат:

In [ ]:
print(decoded_text)

яио обыот а асеио  р с чео пкаовттьлру р с нир с л      ао ьбслилоеонеста пк оио олао   нвноас выер маннй каго ил неио о п нвм нпоео опоонамэй в о н св  уаоно нвяорненигл ноио ое  нолао аненалели йь  со а т ттододо го ил н вм нпоео о ао риклаи гожи т  еи аловапое  соио олао ио о ьлдяр няееватодви оио олао в уяячеоио о вдрда новезт ота   ао очтенеи а и нг ст  счылао и  зх нестз


Результат получился хуже. 

#MCMC-сэмплирование, работающее на статистике биграмм

In [ ]:
n_iter = 20000
verbose = 5000
alphabet = list(ALPHABET)

nan_replacement = 1 / len(ALPHABET) ** 2

def loglikelihood(text):
    bigram_counts = Counter([''.join(text[i: i + 2]) for i in range(len(text) - 2)])
    return np.sum([count * np.log(bigram_freqs.get(bigram, nan_replacement))
                    for bigram, count in bigram_counts.items()])

def check(cur_llh, new_llh):
    if new_llh > cur_llh:
        return True
    return np.random.rand() < np.exp(new_llh - cur_llh)

def decode(text):
    best_decoded_text = copy(text)
    cur_llh = best_llh = loglikelihood(text)
    for iteration in tqdm(range(n_iter)):
        copy_text = copy(text)
        
        letters = np.random.choice(alphabet, 2, replace=False)
        for i in range(len(copy_text)):
            if copy_text[i] == letters[0]:
                copy_text[i] = letters[1]
            elif copy_text[i] == letters[1]:
                copy_text[i] = letters[0]
        
        # new_text = _mutate(copy(text))

        new_llh = loglikelihood(copy_text)
        if check(cur_llh, new_llh):
            text = copy_text
            cur_llh = new_llh
            if cur_llh > best_llh:
                best_llh = cur_llh
                best_decoded_text = copy(text)

    return ''.join(best_decoded_text)

In [ ]:

mcmc = decode(list(encoded_text))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20000/20000 [00:16<00:00, 1215.16it/s]


In [ ]:
print(mcmc)

она улыбнулась на его похвалу и через его плечо продолёала разглядывать залу она была не вновь выезёаъшая у которой на бале все лица сливаътся в одно волжебное впечатление она и не была затасканная по балам девужка которой все лица бала так знакомы что наскучили но она была на середине этих двух  она была возбуёдена а вместе с тем обладала собой настолько что могла наблъдать  


Практически идеально. По крайней мере легко читается

#4. Расшифровка сообщения

In [ ]:
to_decode = "←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏"

In [ ]:
to_decode_unigram_freqs = base_freqs(to_decode)
to_decode_mapping = decoded(unigram_freqs, to_decode_unigram_freqs)
to_decode_text = base_decode(to_decode, to_decode_mapping)

In [ ]:
print(to_decode_text)

олна рд раяасо иевьтнуидг ана пемса иевьтнуидг соклс б шсеые леезжоиач кесевдг ноыке пвемастсу лкевоо рлоые рд рло ляонтна пвтрануие а пенбмасо ьтклаьтнуидг зтнн йт пелнояиоо мосровсео йтятиао кбвлт хесч кеиомие ч иамоые ио езожтю


In [ ]:
mcmc = decode(list(to_decode_text))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20000/20000 [00:10<00:00, 1826.57it/s]


In [ ]:
print(mcmc)

если вы вимите нордальный или почти нордальный текст у этого соожшения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный жалл за послемнее четвертое замание курса ботя конечно я ничего не ожешаю


Достаточно понятный текст, по крайне мере смысл понятен на 100%))

#6. БОНУС

Я сразу вспомнил Юрия Кнорозова, который смог расшифровать письменность Майа. Т.е. этот метод определенно подходит для расшифровки древних языков.
Возможно подобный метод еще подойдет и для расшифровки молекулярных цепочек, так как молекулы формируются по вполне четким и формазиуемым правилам.